In [22]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
%matplotlib inline
% matplotlib inline


pd.options.display.max_columns = 50

In [19]:
data = pd.read_csv('cleaned_dummied_startup_data.csv')
data.drop(['Unnamed: 0', 'founded_at', 'last_funding_at', 'first_funding_at'], axis=1, inplace=True)

In [50]:
status = []

for val in data['status']:
    if val == 'acquired':
        status.append(0)
    elif val == 'operating':
        status.append(1)
    else:
        status.append(2)
data['target'] = status        

data.drop(['status'], axis=1, inplace=True)

In [51]:
training_data = data

In [52]:
training_data = training_data.loc[training_data['funding_total_usd'] != ' -   ']

In [37]:
training_data.funding_total_usd = training_data['funding_total_usd'].astype(int)

/Users/brendanferris/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [58]:
base_logreg = LogisticRegression(random_state=42, max_iter= 10**4, class_weight='balanced')

y_baseline = training_data['target']
X_baseline = training_data.drop('target', axis=1)

X_train , X_test, y_train, y_test = train_test_split(X_baseline, y_baseline, test_size = 0.20, random_state=42)


base_logreg.fit(X_train, y_train)

y_log_default_test = base_logreg.predict(X_test)
y_log_default_train = base_logreg.predict(X_train)

residuals1 = np.abs(y_train - y_log_default_train)
print('Training Data:')
print(pd.Series(residuals1).value_counts())
print(pd.Series(residuals1).value_counts(normalize=True))

Training Data:
1    8308
0    7222
2     446
Name: target, dtype: int64
1    0.520030
0    0.452053
2    0.027917
Name: target, dtype: float64


In [60]:
trainrpt = print(classification_report(y_train, y_log_default_train))

              precision    recall  f1-score   support

           0       0.15      0.74      0.24      1671
           1       0.90      0.43      0.59     13504
           2       0.12      0.15      0.13       801

    accuracy                           0.45     15976
   macro avg       0.39      0.44      0.32     15976
weighted avg       0.78      0.45      0.53     15976



In [56]:
from sklearn.metrics import confusion_matrix

In [59]:
confusion_matrix(y_train, y_log_default_train)

array([[1239,  357,   75],
       [6851, 5863,  790],
       [ 371,  310,  120]])

In [40]:
training_data['status'].unique()

array(['acquired', 'operating', 'closed'], dtype=object)

In [45]:
training_data.dtypes

funding_total_usd        int64
status                  object
seed                   float64
venture                float64
equity_crowdfunding    float64
                        ...   
url_ending_to            int64
url_ending_tv            int64
url_ending_uk            int64
url_ending_us            int64
url_ending_ws            int64
Length: 950, dtype: object

In [47]:
training_data.seed.to_list()

[1750000.0,
 0.0,
 0.0,
 0.0,
 40000.0,
 15000.0,
 0.0,
 420000.0,
 750000.0,
 0.0,
 50000.0,
 0.0,
 0.0,
 69000.0,
 0.0,
 0.0,
 500000.0,
 2200000.0,
 0.0,
 0.0,
 0.0,
 50000.0,
 0.0,
 0.0,
 500000.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 50000.0,
 400000.0,
 0.0,
 50000.0,
 200000.0,
 118000.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 58000.0,
 0.0,
 2275000.0,
 0.0,
 0.0,
 25000.0,
 0.0,
 750000.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 25000.0,
 0.0,
 250000.0,
 0.0,
 1465000.0,
 28000.0,
 0.0,
 350000.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 750000.0,
 0.0,
 0.0,
 0.0,
 0.0,
 50000.0,
 40000.0,
 2000000.0,
 1000000.0,
 0.0,
 0.0,
 0.0,
 25000.0,
 0.0,
 1050000.0,
 0.0,
 0.0,
 800000.0,
 0.0,
 400000.0,
 0.0,
 0.0,
 20900.0,
 0.0,
 0.0,
 0.0,
 0.0,
 50000.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1400000.0,
 525000.0,
 0.0,
 0.0,
 50000.0,
 250000.0,
 0.0,
 0.0,
 0.0,
 0.0,
 2700000.0,
 0.0,
 0.0,
 0.0,
 0.0,
 3800000.0,
 0.0,
 450000.0,
 0.0,
 0.0,
 0.0,
 0.0,
 600000.0,
 0.0,
 0.0,
 

In [48]:
y_train

21271    operating
525       acquired
20742    operating
21189       closed
3328     operating
           ...    
13201       closed
13979    operating
6248     operating
990      operating
18366    operating
Name: status, Length: 15976, dtype: object